In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

In [2]:
import json

In [3]:
# Import API key
from api_keys import audio_api

In [4]:
artists = []

In [5]:
artist_names=[]

In [23]:
ran_ids = np.random.randint(low=100000, high=500000, size=20)
print(ran_ids)

#ran_ids = [112024, 0, 100000, 114364]

[153217 142468 406746 412100 294451 156382 428430 227019 224482 203641
 365340 210788 219634 121746 142325 332437 395108 308507 124392 402990]


In [24]:
# https://theaudiodb.com/api/v1/json/1/artist.php?i=112024
id_url = "https://theaudiodb.com/api/v1/json/1/artist.php?i="

#create a loop that uses random number
for num in ran_ids:
    unique_url = id_url + f'{num}'
    
    #call api url
    request = requests.get(unique_url)
    
    try:
        #convert to json
        info = request.json()
    
    #exception json decode error
    #https://docs.python.org/3/tutorial/controlflow.html
    except json.JSONDecodeError:
        #end this iteration and continue new iteration of for loop
        continue
        
    try:
        #create a variable for single artist to reference later
        artist = info['artists'][0]['strArtist']
        
    #is url responds with {'artists': None}
    except (TypeError):
        continue
        
    if artist not in artist_names:
        
        #populate artists_ids list using info(json)
        #{'artists': [{'idArtist': '114364', 'strArtist': 'Beyoncé', ...
        #dictionary{'artists':['{dictionary}']}
        artist_names.append(info['artists'][0]['strArtist'])
    

    
    print(artist_names)

['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infinity', 'Lisa Crawley', 'Otis Rush', 'Amely Suncroll']
['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infinity', 'Lisa Crawley', 'Otis Rush', 'Amely Suncroll', 'B-Movie']
['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infinity', 'Lisa Crawley', 'Otis Rush', 'Amely Suncroll', 'B-Movie', 'Solar Project']
['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infinity', 'Lisa Crawley', 'Otis Rush', 'Amely Suncroll', 'B-Movie', 'Solar Project', 'Koritni']
['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infinity', 'Lisa Crawley', 'Otis Rush', 'Amely Suncroll', 'B-Movie', 'Solar Project', 'Koritni', 'Les 3 mousquetaires']
['Vox Luminis', 'Drowning Pool', 'Orchestre philharmonique de Radio France', 'Dimness Through Infini

In [ ]:
#convert list to set to remove redundant artists
#alternative is "if not in then .append"
unique_artist_ids = set(artist_ids)
print(unique_artist_ids)


In [ ]:
print(id_url)

In [ ]:
# TEST LOOP TO ADD ARTISTS 

audio_data = {'artist':[],
              'style':[], 
              'genre':[]
             }

#loop over artists dictionary and add data to dictionary
for artist in artists:
    base_url = "http://theaudiodb.com/api/v1/json/1/search.php?s="
            #"http://theaudiodb.com/api/v1/json/1/search.php?s="
    try:
        req = requests.get(base_url + f'{artist}')
        data = req.json()
        #json_str = json.dumps(data)
        json_obj = json.load(json_str)
        
        for item in json_object:
            audio_data.append(json_obj['artist']["strArtist"])

                       
                       #print("The key and value are ({}) = ({})".format(key, value))
            
            #artists.append(["strArtist"])
        
        #for item in data['artists']:   
            #audio_data['artist'].append(item["strArtist"])
            #audio_data['style'].append(item["strStyle"])
            #audio_data['genre'].append(item["strGenre"])
            
        
        #print(base_url + f'{artist}')
        
    except:
        pass

In [ ]:
print(artists)

In [ ]:
json_obj = json.dumps(data)
type(data)

In [ ]:
###THIS ONE IS WORKING

# EXAMPLE theaudiodb.com/api/v1/json/1/search.php?s=coldplay

audio_data = {'artist':[],
              'style':[], 
              'genre':[]
             }

#loop over artists dictionary and add data to dictionary
for artist in artists:
    base_url = "http://theaudiodb.com/api/v1/json/1/search.php?s="
            #"http://theaudiodb.com/api/v1/json/1/search.php?s="
    try:
        req = requests.get(base_url + f'{artist}')
        data = req.json()
        #url = urllib3.urlopen(base_url + f'{artist}')
        #data = json.load(url)
        
        for item in data['artists']:
        #add values to dictionary
            #print(["strArtist"])
            #print(["strStyle"])
            #print(["strGenre"])
        
            audio_data['artist'].append(item["strArtist"])
            audio_data['style'].append(item["strStyle"])
            audio_data['genre'].append(item["strGenre"])
            
            #audio_data['artist'].append(item["artists"]["strArtist"])
            #audio_data['style'].append(item["artists"]["strStyle"])
            #audio_data['genre'].append(item["artists"]["strGenre"])
        
        #print(base_url + f'{artist}')
        
    except:
        pass

In [ ]:
print(audio_data)

In [ ]:
audio_df = pd.DataFrame(audio_data)
print(audio_df)

In [ ]:
#base_url = "http://theaudiodb.com/api/v1/json/"+str(audio_api)+"/search.php?s="
#url = urllib3.urlopen(base_url + f'{artist}')
#data = json.load(url)

#for item in data['artists']:
#    print(item['strArtist'])

In [ ]:
print(base_url)

In [ ]:
#print(audio_api)

In [ ]:
# https://www.youtube.com/watch?v=pxofwuWTs7c
# this turorital has been helpful

In [ ]:
#possible questions
#location and genre
#year formed and genre

#pull artist, location, year formed, genre


In [ ]:
case_list = {}
for entry in data:
    if key in case_list:
        case_list[key1].append(value)
    else:
        case_list[key1] = [value]

In [ ]:
# some JSON:
x =  '{ "name":"John", "age":30, "city":"New York"}'

# parse x:
y = json.dump(data)

# the result is a Python dictionary:
print(y)

In [ ]:
for key, value in data.items():
    print(key, value)

In [ ]:
for item in data['artists']:
    audio_data['artist'].append(item["strArtist"])

In [ ]:
print(audio_data)